## Inverse Kinematics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#Function Definitions
This computes $\vec f(\vec \theta)$

In [ ]:
# compute the forward kinematics
# r is the length of each joint of the arm; this should be a fixed number
# theta is the amount each joint should be rotated
# start is the location of the first/root joint of the arm; this vector needs an extra 1 appended to the end
def f(r, theta, start):
    curPoint = start
    points = start
    T = np.eye(3,3)
    
    # here, we go down the arm rotating the current point and moving it down the length of the joint for each joint
    for i in range(0,r.size):
        # rotation
        X = np.matrix([[np.cos(theta[0,i]),-np.sin(theta[0,i]),0],[np.sin(theta[0,i]),np.cos(theta[0,i]),0],[0,0,1]])
        T = T * X
        # translation
        Z = np.matrix([[1,0,r[0,i]],[0,1,0],[0,0,1]])
        T = T * Z
        # update the current point to the end of the current joint
        curPoint = T * start
        points = np.c_[points,curPoint] # keep track of the set of points so we can plot the arm later
    return points

This computes $\vec g(\vec \theta)$

In [ ]:
# r, theta, and start are the same for f
# target is a vector indicating the target point we want the end of the arm to be at
def g(r, theta, start, target):
    points = f(r, theta, start)
    pos = points[0:2,-1]
    return pos - target

This computes the Jacobian $J_{\vec g}(\vec\theta)$

In [ ]:
# the inputs are the same as g
# this computes the Jacobian matrix numerically
def Jg(r, theta, start, target):
    epsilon = 1e-6
    J = np.zeros((2,0))
    for i in range(0,r.size):
        # use a central difference to estimate the derivatives
        t = theta
        t[0,i] = t[0,i] + epsilon
        p = g(r, t, start, target)
        t = theta
        t[0,i] = t[0,i] - epsilon
        n = g(r, t, start, target)
        J = np.c_[J,(p - n) / (2 * epsilon)]
    return J

This computes the pseudoinverse of the input matrix

In [ ]:
def pseudoinverse(A):
    Ainv = np.zeros((4,2)) #STUDENT, compute the pseudo inverse in this function using the SVD
    return Ainv

This computes the direction we need to move $\vec \theta$ to get closer to the target

In [ ]:
# x is the current point
# c is f(x) where x is the current point
# J is the Jacobian evaluated at the point x
def newton(x, c, J):
    delta = - pseudoinverse(J) * c
    return delta

This updates $\vec \theta$ by adjusting $\eta$ to decide how far to move $\vec \theta$

In [ ]:
# this performs one newton step and estimates eta by starting with eta=1
# if the end effector of the arm is moved farther away after this step,
# we divide eta by 2 and then repeat until we move closer or if eta is too small
def update(theta, r, start, target):
    cost = g(r, theta, start, target)
    oldCost = np.linalg.norm(cost)
    J = Jg(r, theta, start, target)
    delta = newton(theta.T, cost, J).T
    eta = 1.0
    while eta > 1e-4:
        cost = g(r, theta + eta * delta, start, target)
        temp = np.linalg.norm(cost)
        if temp < oldCost:
            theta = theta + eta * delta
            newCost = temp
            break
        eta = eta / 2
    return theta

This puts everything together to compute $\vec \theta$ that moves the arm to the target

In [ ]:
def IK(thetaStart, r, start, target):
    theta = thetaStart
    oldCost = np.linalg.norm(g(r, theta, start, target))
    newCost = oldCost
    iteration = 0
    while (newCost < oldCost or iteration < 1) and iteration < 8:
        oldCost = newCost
        theta = update(theta, r, start, target)
        newCost = np.linalg.norm(g(r, theta, start, target))
        iteration = iteration + 1
    return theta

This is a helper function to visualize the arm

In [ ]:
def plotArm(r, theta, start, target):
    p = f(r, theta, start)
    fig = plt.figure()
    plt.xlim(-4, 7)
    plt.ylim(-4, 7)
    plt.plot(target[0,:].T,target[1,:].T,'b-o',markersize=12)
    plt.plot(p[0,:].T,p[1,:].T,'r-o',linewidth=4,markersize=8)

#Testing

This tests your pseudo inverse function in three cases

In [ ]:
%matplotlib nbagg

# Define the location of the first arm joint and the length of each joint
r = np.matrix([[1.0,1.0,1.0,1.0]])
start = np.matrix([[0],[0],[1]]) # The one at the end is there for homogeneous coordinates

# Test if the arm can reach a target
theta = np.matrix([[-0.16227303, -0.60693046, 0.72069313, 0.70009674]])
target = np.matrix([[2.0],[1.6]])
theta = IK(theta, r, start, target)
plotArm(r, theta, start, target)

# Test if the arm can point to a target out of reach
theta = np.matrix([[0.0,0.0,0.0,0.0]])
target = np.matrix([[4.0],[4.0]])
theta = IK(theta, r, start, target)
plotArm(r, theta, start, target)

# Test how well the pseudoinverse works with small singular values
theta = np.matrix([[0.0,0.0,0.0,0.0]]) + (np.random.rand(1,4) - 0.5) / 1e6
start = np.matrix([[0],[0],[1]])
target = np.matrix([[5.0],[2.0]])
theta = IK(theta, r, start, target)
plotArm(r, theta, start, target)

This animates the arm reaching for a moving target.  When the target goes out of reach, the arm should point towards the target because that is the closest it can get to the target.

In [ ]:
# initialize the arm
theta = np.matrix([[0.0,0.0,0.0,0.0]])
r = np.matrix([[1.0,1.0,1.0,1.0]])
start = np.matrix([[0],[0],[1]])

# set up the plot
fig = plt.figure()
ax = plt.axes(xlim=(-4, 4), ylim=(-4, 4), aspect='equal')
p = f(r, theta, start)
plt.xlim(-1, 4)
plt.ylim(-5, 5)
line, = plt.plot([], [], 'r-o', linewidth=4, markersize=10)
point, = plt.plot([],[],'b-o',markersize=12)
angle = 2.0

def update_arm(num, r, s):
    global angle, theta
    angle = np.mod(angle + 0.05,2 * np.pi)
    t = np.matrix([[2.0],[4.5 * np.sin(angle)]])
    theta = IK(theta, r, s, t)
    theta = np.mod(theta, 2 * np.pi)
    p = f(r, theta, s)
    line.set_data(p[0:2,:])
    point.set_data(t)

arm_ani = animation.FuncAnimation(fig, update_arm, 200, fargs=(r, start),
    interval=1000/12, blit=True)